Goal of this step1 is to ingest the documents and spilt them into chunks , embedded them and store them into a vector database eg (FAISS : Facebook AI Similarity Search)

In [5]:
#Installing the lib and importing them

!pip install -qU langchain cohere


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: C:\Users\sri19\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
!pip install -qU "faiss-cpu" "tiktoken" "PyPDF2"
!pip install  -qU "sentence-transformers" "python-dotenv"


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: C:\Users\sri19\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: C:\Users\sri19\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Setting up API key 

In [37]:
import os
from dotenv import load_dotenv
load_dotenv()


True

In [ ]:
!pip install cryptography


Loading the PDF / Documents

In [32]:
import json
from pathlib import Path
from langchain.schema import Document
from langchain.document_loaders import PyPDFLoader

documents = []

# === 1. Load from arXiv JSON ===
json_path = Path("../../archive/arxiv-metadata-oai-snapshot.json")
if json_path.exists():
    with open(json_path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            if i >= 100:  
                break
            entry = json.loads(line)
            content = f"Title: {entry.get('title','').strip()}\n\nAbstract: {entry.get('abstract','').strip()}\n\nCategories: {entry.get('categories','').strip()}"
            metadata = {
                "arxiv_id": entry.get("id", ""),
                "authors": entry.get("authors", ""),
                "update_date": entry.get("update_date", "")
            }
            documents.append(Document(page_content=content, metadata=metadata))
    print(f" Loaded {len(documents)} documents from arXiv JSON")
else:
    print(" arXiv JSON not found")

# === 2. Load PDFs ===
pdf_path = Path("../Papers")
if pdf_path.exists():
    pdf_files = list(pdf_path.glob("*.pdf"))
    for file in pdf_files:
        loader = PyPDFLoader(str(file))
        pdf_docs = loader.load()
        documents.extend(pdf_docs)
    print(f" Loaded {len(pdf_files)} PDFs with {len(documents)} total documents")
else:
    print(" PDF folder not found")


 Loaded 100 documents from arXiv JSON
 Loaded 2 PDFs with 119 total documents


Now splitting the Document into chunks

In [34]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Small
small_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
small_chunks = small_splitter.split_documents(documents)

# Large
large_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
large_chunks = large_splitter.split_documents(documents)

# Prepare texts for embedding
small_texts = [chunk.page_content for chunk in small_chunks]
large_texts = [chunk.page_content for chunk in large_chunks]

print(f" {len(small_chunks)} small chunks, {len(large_chunks)} large chunks.")


# In the above we need overlap so that we do not miss the context while using the chunks and I am using samll_chunks for facts and large_chunks for summerization task





 676 small chunks, 221 large chunks.


## Embedding  chunks first 

In [35]:
from tqdm import tqdm
import time

def embed_in_batches(texts, batch_size=96, input_type="search_document"):
    all_embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding"):
        batch = texts[i:i+batch_size]
        try:
            response = co.embed(
                texts=batch,
                model="embed-english-v3.0",
                input_type=input_type
            )
            all_embeddings.extend(response.embeddings)
        except Exception as e:
            print(f" Batch {i // batch_size + 1} failed: {e}")
            time.sleep(5)
    return all_embeddings

# Run for small & large
small_embeddings = embed_in_batches(small_texts)
print("Embedded", len(small_embeddings), "small chunks")

large_embeddings = embed_in_batches(large_texts)
print("Embedded", len(large_embeddings), "large chunks")


Embedding:  38%|███▊      | 3/8 [00:01<00:02,  1.83it/s]

 Batch 4 failed: status_code: 502, body: 
<html><head>
<meta http-equiv="content-type" content="text/html;charset=utf-8">
<title>502 Server Error</title>
</head>
<body text=#000000 bgcolor=#ffffff>
<h1>Error: Server Error</h1>
<h2>The server encountered a temporary error and could not complete your request.<p>Please try again in 30 seconds.</h2>
<h2></h2>
</body></html>



Embedding: 100%|██████████| 8/8 [02:32<00:00, 19.02s/it]


Embedded 580 small chunks


Embedding: 100%|██████████| 3/3 [00:01<00:00,  1.52it/s]

Embedded 221 large chunks


## Saving Small and Large Embeddings to FAISS

In [ ]:
from langchain.vectorstores import FAISS
from langchain_core.embeddings import Embeddings
from langchain.schema import Document

# Wrap embeddings
class WorkingCohereEmbeddings(Embeddings):
    def embed_documents(self, texts):
        return co.embed(texts=texts, model="embed-english-v3.0", input_type="search_document").embeddings
    def embed_query(self, text):
        return self.embed_documents([text])[0]

embedding_model = WorkingCohereEmbeddings()

# Convert texts to Documents
small_docs = [Document(page_content=text) for text in small_texts]
large_docs = [Document(page_content=text) for text in large_texts]

# Save to FAISS
faiss_small = FAISS.from_documents(small_docs, embedding_model)
faiss_small.save_local("vectorstore/faiss_small", index_name="small_index")

faiss_large = FAISS.from_documents(large_docs, embedding_model)
faiss_large.save_local("vectorstore/faiss_large", index_name="large_index")

print(" Both small and large FAISS indexes saved.")


In [39]:
# Embedding wrapper

from langchain_core.embeddings import Embeddings

class WorkingCohereEmbeddings(Embeddings):
    def embed_documents(self, texts):
        response = co.embed(
            texts=texts,
            model="embed-english-v3.0",
            input_type="search_document"
        )
        return response.embeddings

    def embed_query(self, text):
        return self.embed_documents([text])[0]

embedding_model = WorkingCohereEmbeddings()


In [ ]:
from langchain.vectorstores import FAISS

# Save small chunk index
faiss_small = FAISS.from_documents(small_docs, embedding_model)
faiss_small.save_local("vectorstore/faiss_small", index_name="small_index")
print("Saved small chunks to FAISS at vectorstore/faiss_small")

# Save large chunk index
faiss_large = FAISS.from_documents(large_docs, embedding_model)
faiss_large.save_local("vectorstore/faiss_large", index_name="large_index")
print(" Saved large chunks to FAISS at vectorstore/faiss_large")


## Load the embeddings

In [41]:
# Small chunks for factual queries
vector_store_small = FAISS.load_local(
    folder_path="vectorstore/faiss_small",
    index_name="small_index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

# Large chunks for summary/reasoning queries
vector_store_large = FAISS.load_local(
    folder_path="vectorstore/faiss_large",
    index_name="large_index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

print(" Both FAISS indexes loaded.")


 Both FAISS indexes loaded.


## Retreival

In [42]:
# Use k=3 for top-3 most similar chunks
retriever_small = vector_store_small.as_retriever(search_kwargs={"k": 3})
retriever_large = vector_store_large.as_retriever(search_kwargs={"k": 3})


## Creating RAG Chains

In [43]:
from langchain.chains import RetrievalQA
from langchain.llms import Cohere as CohereLLM

llm = CohereLLM(cohere_api_key=api_key)

# Chain for factual Q&A
rag_chain_small = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever_small,
    return_source_documents=True
)

# Chain for summary-level reasoning
rag_chain_large = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever_large,
    return_source_documents=True
)


In [45]:
query = "Do you have a categories for realted to memory and performance?"
result = rag_chain_small.invoke(query)
print("🔹 Answer:\n", result["result"])


🔹 Answer:
  I don't have a specific category or topic named 'memory and performance', but I do have categories related to cognitive computing and machine learning, which may be of interest to you. These include: 

- quant-ph: This category focuses on quantum physics, including quantum information and computation, which may have implications for advanced computing and memory systems. 

- cs.IT and math.IT: These categories cover computational intelligence and mathematical modeling, which are often applied to developing innovative computing systems and optimizing performance. 

- cond-mat.stat-mech and cond-mat.mtrl-sci: These categories concern the study of matter and materials behavior, including statistical mechanics and condensed matter physics, which could involve research related to memory storage or performance. 

If you have a specific question related to memory or performance, please let me know, and I'll do my best to answer it or guide you to additional resources. 
